In [1]:
import numpy as np
import pandas as pd

In [2]:
data = {(1001400,1001402,9.7),
(1001402,1001404,20.2),
(1001404,1001406,16),
(1001406,1001408,21.7),
(1001408,1001410,11.1),
(1001410,1001412,15.6),
(1001412,1001414,18.2),
(1001414,1001416,13.2),
(1001416,1001418,13.6),
(1001418,1001420,12.9),
(1001420,1001422,9.6),
(1001422,1001424,11.4),
(1001424,1001426,18.6),
(1001426,1001428,15.8),
(1001428,1001430,8.6),
(1001430,1001432,9),
(1001432,1001434,7.9),
(1001434,1001436,4),
(1001436,1001438,9),
(1001436,1001437,5),
(1001438,1001437,4),
(1001438,1001440,10),
(1001440,1001442,3.9),
(1001442,1001488,4.5),
(1001488,1004356,4),
(1004356,1004354,2),
(1004354,1004355,2),
(1004355,1001444,0.7),
(1001444,1001446,6.6),
(1001446,1001448,9.6),
(1001448,1001450,15.7),
(1001450,1001452,9.9),
(1001452,1001454,11.3),
(1001454,1001456,13.6),
(1001456,1001458,8.9),
(1001458,1001460,5.1),
(1001460,1001461,12.8),
(1001460,1001462,17.9),
(1001461,1001462,5.1),
(1001462,1001464,26.7),
(1001464,1001466,8.5),
(1001466,1001468,10.7),
(1001468,1001470,10.6),
(1001470,1001472,16)}

In [3]:
df = pd.DataFrame(data = data,columns=['id_start','id_end','distance'])

In [4]:
df.head()

,id_start,id_end,distance
0,1001436,1001438,9.0
1,1001406,1001408,21.7
2,1001452,1001454,11.3
3,1001404,1001406,16.0
4,1001456,1001458,8.9


### Python Task 2
### Question 1: Distance Matrix Calculation

In [5]:
#Create a function named calculate_distance_matrix that takes the dataset-3.csv as input and generates a DataFrame representing 
#distances between IDs.

#The resulting DataFrame should have cumulative distances along known routes, with diagonal values set to 0. If distances 
#between toll locations A to B and B to C are known, then the distance from A to C should be the sum of these distances. 
#Ensure the matrix is symmetric, accounting for bidirectional distances between toll locations (i.e. A to B is equal to B to A).

In [17]:
def calculate_distance_matrix(file_name):
   
    
    ids = pd.unique(df[['ID1', 'ID2']].values.ravel('K'))
    
    distance_matrix = pd.DataFrame(np.inf, index=ids, columns=ids)
 
    np.fill_diagonal(distance_matrix.values, 0)

    for index, row in df.iterrows():
        distance_matrix.loc[row['ID1'], row['ID2']] = row['Distance']
        distance_matrix.loc[row['ID2'], row['ID1']] = row['Distance']
    

    for k in ids:
        for i in ids:
            for j in ids:
                if distance_matrix.loc[i, j] > distance_matrix.loc[i, k] + distance_matrix.loc[k, j]:
                    distance_matrix.loc[i, j] = distance_matrix.loc[i, k] + distance_matrix.loc[k, j]
                    distance_matrix.loc[j, i] = distance_matrix.loc[i, j]
    
    return distance_matrix

### Question 2: Unroll Distance Matrix

In [7]:
#Create a function unroll_distance_matrix that takes the DataFrame created in Question 1. 
#The resulting DataFrame should have three columns: columns id_start, id_end, and distance.

#All the combinations except for same id_start to id_end must be present in the rows with their distance values 
#from the input DataFrame.

In [8]:
def unroll_distance_matrix(df):
    result = pd.DataFrame(columns=['id_start', 'id_end', 'distance'])

    for i in range(len(df)):
       
        for j in range(len(df.columns)):
            
            if i != j:
               
                result = result.append({'id_start': df.index[i], 'id_end': df.columns[j], 'distance': df.iloc[i, j]}, 
                                       ignore_index=True)

    return result
df_2 = unroll_distance_matrix(df)

C:\Users\Asus\AppData\Local\Temp\ipykernel_7448\3720160476.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'id_start': df.index[i], 'id_end': df.columns[j], 'distance': df.iloc[i, j]},
C:\Users\Asus\AppData\Local\Temp\ipykernel_7448\3720160476.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'id_start': df.index[i], 'id_end': df.columns[j], 'distance': df.iloc[i, j]},
C:\Users\Asus\AppData\Local\Temp\ipykernel_7448\3720160476.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'id_start': df.index[i], 'id_end': df.columns[j], 'distance': df.iloc[i, j]},
C:\Users\Asus\AppData\Local\Temp\ipykernel_7448\3720160476.py:10: FutureWarning: The frame.

In [9]:
df_2

,id_start,id_end,distance
0,0,id_end,1001438
1,0,distance,9.0
2,1,id_start,1001406
3,1,distance,21.7
4,2,id_start,1001452
...,...,...,...
124,42,id_end,1004356
125,42,distance,4.0
126,43,id_start,1001402
127,43,id_end,1001404


### Question 3: Finding IDs within Percentage Threshold

In [10]:
#Create a function find_ids_within_ten_percentage_threshold that takes the DataFrame created in Question 2 and a 
#reference value from the id_start column as an integer.

#Calculate average distance for the reference value given as an input and return a sorted list of values from id_start 
#column which lie within 10% (including ceiling and floor) of the reference value's average.

In [11]:
def find_ids_within_ten_percentage_threshold(df, ref_value):
    
    avg_distance = df[df['id_start'] == ref_value]['distance'].mean()

    lower_threshold = avg_distance * 0.9
    upper_threshold = avg_distance * 1.1

    filtered_df = df[(df['distance'] >= lower_threshold) & (df['distance'] <= upper_threshold)]

    return sorted(filtered_df['id_start'].unique())

### Question 4: Calculate Toll Rate

In [13]:
#Create a function calculate_toll_rate that takes the DataFrame created in Question 2 as input and calculates toll rates 
#based on vehicle types.

#The resulting DataFrame should add 5 columns to the input DataFrame: moto, car, rv, bus, and truck with their respective 
#rate coefficients. The toll rates should be calculated by multiplying the distance with the given rate coefficients for each 
#vehicle type:

#0.8 for moto
#1.2 for car
#1.5 for rv
#2.2 for bus
#3.6 for truck

In [14]:
def calculate_toll_rate(df):
    
    rate_coefficients = {
        'moto': 0.8,
        'car': 1.2,
        'rv': 1.5,
        'bus': 2.2,
        'truck': 3.6
    }

    for vehicle_type, rate_coefficient in rate_coefficients.items():
        df[vehicle_type] = df['distance'] * rate_coefficient

    return df

### Question 5: Calculate Time-Based Toll Rates

In [15]:
#Create a function named calculate_time_based_toll_rates that takes the DataFrame created in Question 3 as input and 
#calculates toll rates for different time intervals within a day.

#The resulting DataFrame should have these five columns added to the input: start_day, start_time, end_day, and end_time.

#start_day, end_day must be strings with day values (from Monday to Sunday in proper case)
#start_time and end_time must be of type datetime.time() with the values from time range given below.
#Modify the values of vehicle columns according to the following time ranges:

#Weekdays (Monday - Friday):

#From 00:00:00 to 10:00:00: Apply a discount factor of 0.8
#From 10:00:00 to 18:00:00: Apply a discount factor of 1.2
#From 18:00:00 to 23:59:59: Apply a discount factor of 0.8
#Weekends (Saturday and Sunday):

#Apply a constant discount factor of 0.7 for all times.
#For each unique (id_start, id_end) pair, cover a full 24-hour period (from 12:00:00 AM to 11:59:59 PM) and span all 7 days 
#of the week (from Monday to Sunday).

#Sample result dataframe: